# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\mathrm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\mathrm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\mathrm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [4]:
#第6章のデータを使用した．（Google Newsデータセット）
from gensim.models import KeyedVectors
import numpy as np
from dotenv import load_dotenv
import os

load_dotenv()
model_path = os.getenv("GoogleNews_Vector")
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

vocab_size = len(model.key_to_index)
emb_dim = model.vector_size

word_to_id = {"<PAD>":0}
id_to_word = {0:"<PAD>"}

embedding_matrix = np.zeros((vocab_size+1, emb_dim))

for word, gensim_id in model.key_to_index.items():
    word_id = gensim_id + 1
    word_to_id[word] = word_id
    id_to_word[word_id] = word
    embedding_matrix[word_id] = model[word]
    
print(embedding_matrix.shape)
print(f"|V|:{embedding_matrix.shape[0]}")
print(f"d_emb:{embedding_matrix.shape[1]}")

(3000001, 300)
|V|:3000001
d_emb:300


## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [5]:
import csv
import torch

def load_sst2(file_path, word_to_id):
    data = []
    with open(file_path, encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t")
        for row in reader:
            text = row["sentence"]
            label = float(row["label"])

            words = text.split()
            input_ids = [word_to_id[w] for w in words if w in word_to_id]

            if len(input_ids) == 0:
                continue

            data.append({
                "text": text,
                "label": torch.tensor([label]),
                "input_ids": torch.tensor(input_ids)
            })
    return data

train_path = "../ch07/data/SST-2/train.tsv"
dev_path= "../ch07/data/SST-2/dev.tsv"

train_data = load_sst2(train_path, word_to_id)
dev_data = load_sst2(dev_path, word_to_id)

# print(f"train_data: {len(train_data)}")
# print(f"dev_data: {len(dev_data)}")
print(train_data[0])

{'text': 'hide new secretions from the parental units ', 'label': tensor([0.]), 'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])}


## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [6]:
import torch.nn as nn

class AvgvecEmbclassification(nn.Module):
    def __init__(self, embedding):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding,freeze=True,padding_idx=0)
        self.linear = nn.Linear(embedding.size(1), 1)

    def forward(self, input_ids):
        embeds = self.embedding(input_ids)   
        sent_vec = embeds.mean(dim=0)         
        logit = self.linear(sent_vec)         
        prob = torch.sigmoid(logit)
        return prob

## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [7]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_tensor = torch.FloatTensor(embedding_matrix)
model = AvgvecEmbclassification(embedding_tensor).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.linear.parameters(), lr=1e-3)

def train_eval(model,data):
    model.train()
    total_loss = 0
    for sample in data:
        input_ids = sample["input_ids"].to(device)
        label = sample["label"].to(device)

        optimizer.zero_grad()
        prob = model(input_ids)
        loss = criterion(prob, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
    avg_loss = total_loss / len(data)
    return avg_loss

def evaluate(model, data):
    model.eval()
    correct = 0
    with torch.no_grad():
        for sample in data:
            input_ids = sample["input_ids"].to(device)
            label = sample["label"].to(device)

            prob = model(input_ids)
            pred = (prob >= 0.5).float()
            if pred.item() == label.item():
                correct += 1
    accuracy = correct / len(data)
    return accuracy

best_acc = 0

num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_eval(model, train_data)
    dev_acc = evaluate(model, dev_data)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Dev Accuracy: {dev_acc:.4f}")
    
    if dev_acc > best_acc:
        best_acc = dev_acc
        torch.save(model.state_dict(), "../ch08/model/73_model.pt")

Epoch 1/5, Train Loss: 0.4022, Dev Accuracy: 0.7936
Epoch 2/5, Train Loss: 0.3725, Dev Accuracy: 0.8005
Epoch 3/5, Train Loss: 0.3701, Dev Accuracy: 0.7982
Epoch 4/5, Train Loss: 0.3692, Dev Accuracy: 0.7993
Epoch 5/5, Train Loss: 0.3688, Dev Accuracy: 0.7993


## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [8]:
model = AvgvecEmbclassification(embedding_tensor).to(device)
model.load_state_dict(torch.load("../ch08/model/73_model.pt",weights_only=True))
model.eval()

dev_acc = evaluate(model, dev_data)
print(f"best_dev_acc: {dev_acc:.4f}")


best_dev_acc: 0.8005


・推奨
/tmp/ipykernel_596915/2353182659.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../ch08/model/73_model.pt"))

## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


In [9]:
def collate(batch):
    batch = sorted(batch, key=lambda x:len(x["input_ids"]),reverse=True)
    max_len=len(batch[0]["input_ids"])
    
    padded_ids = []
    labels = []
    
    for sample in batch:
        ids = sample["input_ids"]
        padding_len = max_len - len(ids)
        
        padding = torch.cat([ids, torch.zeros(padding_len,dtype=torch.long)])
        padded_ids.append(padding.unsqueeze(0))
        labels.append(sample["label"].unsqueeze(0))
        result  = {"input_ids": torch.cat(padded_ids,dim=0),"labels": torch.cat(labels,dim=0)}
        return result

## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [10]:
from torch.utils.data import DataLoader
import torch.optim as optim

batch_size = 32

train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,collate_fn=collate)
dev_loader = DataLoader(dev_data,batch_size=batch_size,shuffle=False,collate_fn=collate)

class AvgvecEmbclassification_batch(nn.Module):
    def __init__(self, embedding):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding,freeze=True,padding_idx=0)
        self.linear = nn.Linear(embedding.size(1), 1)

    def forward(self, input_ids):
        embeds = self.embedding(input_ids)   
        sent_vec = embeds.mean(dim=1)         
        logit = self.linear(sent_vec)         
        prob = torch.sigmoid(logit)
        return prob

def train_one_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        prob = model(input_ids)
        loss = criterion(prob, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(data_loader)

def evaluate(model,loader,device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            
            prob = model(input_ids)
            preds = (prob >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
    return correct / total

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_tenser = torch.FloatTensor(embedding_matrix)
model = AvgvecEmbclassification_batch(embedding_tenser).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.linear.parameters(), lr=1e-3)

num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    dev_acc = evaluate(model, dev_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Dev Accuracy: {dev_acc:.4f}")

    if dev_acc > best_acc:
        best_acc = dev_acc
        torch.save(model.state_dict(), "../ch08/model/76_model.pt")


Epoch 1/10, Train Loss: 0.6647, Dev Accuracy: 0.6786
Epoch 2/10, Train Loss: 0.6178, Dev Accuracy: 0.7500
Epoch 3/10, Train Loss: 0.5873, Dev Accuracy: 0.7143
Epoch 4/10, Train Loss: 0.5682, Dev Accuracy: 0.8214
Epoch 5/10, Train Loss: 0.5440, Dev Accuracy: 0.8214
Epoch 6/10, Train Loss: 0.5257, Dev Accuracy: 0.8214
Epoch 7/10, Train Loss: 0.5173, Dev Accuracy: 0.8214
Epoch 8/10, Train Loss: 0.5139, Dev Accuracy: 0.8214
Epoch 9/10, Train Loss: 0.4940, Dev Accuracy: 0.8214
Epoch 10/10, Train Loss: 0.4920, Dev Accuracy: 0.8214


## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

76.と同様．

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

76の問題との主な違い：  
・embeddingを"freeze=True"から"freeze=False"に変更する．  
・optimizerにmodel.parameters()を渡す．

In [11]:
from torch.utils.data import DataLoader
import torch.optim as optim

batch_size = 32

train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,collate_fn=collate)
dev_loader = DataLoader(dev_data,batch_size=batch_size,shuffle=False,collate_fn=collate)

class AvgvecEmbclassification_batch(nn.Module):
    def __init__(self, embedding):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding,freeze=False,padding_idx=0)
        self.linear = nn.Linear(embedding.size(1), 1)

    def forward(self, input_ids):
        embeds = self.embedding(input_ids)   
        sent_vec = embeds.mean(dim=1)         
        logit = self.linear(sent_vec)         
        prob = torch.sigmoid(logit)
        return prob

def train_one_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        prob = model(input_ids)
        loss = criterion(prob, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(data_loader)

def evaluate(model,loader,device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            
            prob = model(input_ids)
            preds = (prob >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
    return correct / total

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_tenser = torch.FloatTensor(embedding_matrix)
model = AvgvecEmbclassification_batch(embedding_tenser).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 5

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    dev_acc = evaluate(model, dev_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Dev Accuracy: {dev_acc:.4f}")

    if dev_acc > best_acc:
        best_acc = dev_acc
        torch.save(model.state_dict(), "../ch08/model/78_model.pt")


Epoch 1/5, Train Loss: 0.5359, Dev Accuracy: 0.7143
Epoch 2/5, Train Loss: 0.2744, Dev Accuracy: 0.7143
Epoch 3/5, Train Loss: 0.1917, Dev Accuracy: 0.7500
Epoch 4/5, Train Loss: 0.1707, Dev Accuracy: 0.7857
Epoch 5/5, Train Loss: 0.1322, Dev Accuracy: 0.7500


## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。

In [13]:
import torch.nn as nn
import torch

class CNN(nn.Module):
    def __init__(self, embedding, num_filters=100, kernel_size=3):
        super().__init__()

        self.embedding = nn.Embedding.from_pretrained(embedding,freeze=False,padding_idx=0)
        
        self.conv = nn.Conv1d(in_channels=embedding.size(1), out_channels=num_filters, kernel_size=kernel_size)
        self.fc = nn.Linear(num_filters, 1)
        
    def forward(self, input_ids):
        embeds = self.embedding(input_ids)
        x = embeds.transpose(1, 2)
        x = torch.relu(self.conv(x))
        x = torch.max(x, dim=2).values
        logit = self.fc(x)
        prob = torch.sigmoid(logit)
        return prob
   
    
batch_size = 16

train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,collate_fn=collate)
dev_loader = DataLoader(dev_data,batch_size=batch_size,shuffle=False,collate_fn=collate)

def train_one_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        prob = model(input_ids)
        loss = criterion(prob, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(data_loader)

def evaluate(model,loader,device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            
            prob = model(input_ids)
            preds = (prob >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
    return correct / total

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_tenser = torch.FloatTensor(embedding_matrix)
model = CNN(embedding_tenser).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 5

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    dev_acc = evaluate(model, dev_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Dev Accuracy: {dev_acc:.4f}")

    if dev_acc > best_acc:
        best_acc = dev_acc
        torch.save(model.state_dict(), "../ch08/model/79_model.pt")



Epoch 1/5, Train Loss: 0.3916, Dev Accuracy: 0.8000
Epoch 2/5, Train Loss: 0.2074, Dev Accuracy: 0.7455
Epoch 3/5, Train Loss: 0.1743, Dev Accuracy: 0.7818
Epoch 4/5, Train Loss: 0.1423, Dev Accuracy: 0.6909
Epoch 5/5, Train Loss: 0.1341, Dev Accuracy: 0.7091
